In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
import math

import numpy as np
import pandas as pd

from backtesting import Backtest, Strategy
from backtesting.test import SMA
import tulipy as tu


In [11]:
dataset_path = './data/HistoricalData_NG.csv'

In [12]:
def get_adx(data: pd.DataFrame, n: int):
    dx = tu.adx(data.High, data.Low, data.Close, n)
    return return_indicator(dx, len(data))

def get_atr(data: pd.DataFrame, n: int):
    atr = tu.atr(data.High, data.Low, data.Close, n)
    return return_indicator(atr, len(data))

def return_indicator(values, data_len: int):
    a = np.empty((data_len - len(values),))
    a[:] = np.nan
    return np.concatenate([a, values])

In [13]:
class MovingAvg3LineCross(Strategy):

    n1 = 9
    n2 = 18
    n3 = 60
    risk = 1

    def init(self):
        super(MovingAvg3LineCross, self).init()
        self.trailingATRLen = 5
        self.lookback = 3
        self.atrStopMultiplier = 1
        self.shortSelling = False # allowance of short selling
        self.pointvalue = 0.001

        self.fastSMA = self.I(SMA, self.data.Close, self.n1)
        self.medSMA = self.I(SMA, self.data.Close, self.n2)
        self.slowSMA = self.I(SMA, self.data.Close, self.n3)
        self.atr = self.I(get_atr, self.data, 14)
        self.trailingAtr = self.I(get_atr, self.data, 5)
        self.adx = self.I(get_adx, self.data, 14)


    def next(self):
        if len(self.trades) == 0:
            if self.fastSMA[-1] > self.medSMA[-1] > self.slowSMA[-1] and \
                    self.adx[-1] > 20:
                self.buy(size=self.get_position_size())
            if self.fastSMA[-1] < self.medSMA[-1] < self.slowSMA[-1] \
                    and self.shortSelling and self.adx > 20:
                self.sell(size=self.get_position_size())
        else:
            if self.trades[0].is_long:
                if self.data.Low[-1] <= self.get_trailing_stop(long=True):
                    self.position.close()
            else:
                if self.data.High[-1] >= self.get_trailing_stop(long=False) and self.shortSelling:
                    self.position.close()

    def get_position_size(self):
        return math.floor((self.risk/100 * self.equity)/(self.atr * self.pointvalue))

    def get_trailing_stop(self, long: bool):
        if long:
            return np.max(self.data.Low[-3:]) - self.trailingAtr[-1] * self.atrStopMultiplier
        else:
            return np.min(self.data.High[-3:]) + self.trailingAtr[-1] * self.atrStopMultiplier


In [14]:
data = pd.read_csv(dataset_path, sep=',', index_col=0, parse_dates=True)
data = data.dropna()

data = data.set_index(pd.to_datetime(data.index))
data.sort_values(by='Date', inplace=True)

In [15]:
bt = Backtest(data, MovingAvg3LineCross, cash=100_000, commission=0.02, trade_on_close=True)
stats = bt.run()

stats

Start                     2011-08-31 00:00:00
End                       2021-09-14 00:00:00
Duration                   3667 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                     100000.0
Equity Peak [$]                      100000.0
Return [%]                                0.0
Buy & Hold Return [%]               29.748397
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [16]:
bt.plot()

Row(id='2138', ...)